In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用Cloud Function进行在线预测的AutoML图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_classification_online_proxy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_classification_online_proxy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建图像分类模型，并使用Google Cloud的AutoML进行在线预测。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Flowers数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。 在本教程中，您将使用存储在公共云存储桶中的数据集版本。 训练的模型可以预测图像所属的花卉类型，包括五种花：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建并部署一个AutoML图像分类模型，以便进行在线预测。您也可以选择使用`gcloud`命令行工具或通过Google Cloud控制台在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个提供`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的`google-cloud-storage`库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可以的话，请确保在 GPU 运行时下运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的 Google Cloud 项目

**不论你的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建账户时，你会获得 $300 的免费信用额度以用于计算/存储成本。

2. [确保你的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex APIs 和 Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)。

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud 笔记本中。

5. 在下面的单元格中输入你的项目 ID。然后运行这个单元格以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 将以 `!` 开头的行作为 shell 命令运行，并将以 `$` 开头的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲： `us-central1`
- 欧洲： `europe-west4`
- 亚太： `asia-east1`

您可能不能在Vertex中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在进行实时教程会话，则可能正在使用共享测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###认证您的Google Cloud帐号

**如果您正在使用Google Cloud Notebook**，则您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth授权您的帐号。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**将此服务帐号授权给项目**部分，点击角色下拉列表。在过滤框中键入“Vertex”，并选择**Vertex管理员**。在过滤框中键入“Storage Object Admin”，并选择**Storage Object Admin**。

点击创建。将包含密钥的JSON文件下载到本地环境。

在下面的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

在我们的Python环境中导入Vertex客户端库。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常数

为顶点设置以下常数:

- `API_ENDPOINT`: 顶点API服务的终端点，用于数据集、模型、作业、流水线和终点服务。
- `PARENT`: 顶点位置的根路径，用于数据集、模型、作业、流水线和终点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML 常数

设置仅适用于 AutoML 数据集和训练的常数：

- 数据集模式：告诉“数据集”资源服务数据集的类型是什么。
- 数据标记（注释）模式：告诉“数据集”资源服务数据如何被标记（注释）。
- 数据集训练模式：告诉“管道”资源服务要为哪个任务（例如分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml"

# 教程

现在你已经准备好开始创建自己的AutoML图像分类模型了。

## 设置客户端

Vertex 客户端库采用客户端/服务器模型。在您的一侧（Python 脚本）中，您将创建一个客户端，用于向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 用于 `Dataset` 资源的 Dataset 服务。
- 用于 `Model` 资源的 Model 服务。
- 用于训练的 Pipeline 服务。
- 用于部署的 Endpoint 服务。
- 用于提供服务的 Prediction 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记好的数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。此函数会执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的易于阅读的名称。
 - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法 `create_dataset`，具有以下参数：
 - `parent`：您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 根路径。
 - `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法会返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用。虽然这一步通常很快，但在项目中首次使用时，由于需要进行资源分配，可能会有较长的延迟。

您可以使用 `operation` 对象来获取操作（例如创建 `Dataset` 资源）的状态，或者通过调用操作方法来取消操作：

| 方法        | 描述                 |
| ----------- | -------------------- |
| result()    | 等待操作完成并返回 JSON 格式的结果对象。 |
| running()   | 返回操作是否仍在运行的 True/False 值。   |
| done()      | 返回操作是否已完成的 True/False 值。     |
| canceled()  | 返回操作是否已取消的 True/False 值。     |
| cancel()    | 取消操作（可能需要最多 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("flowers-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例对应的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

Vertex中用于图像的`Dataset`资源对您的数据有一些要求：

- 图像必须存储在云存储桶中。
- 每个图像文件必须是图像格式（PNG，JPEG，BMP，...）。
- 您的云存储桶中必须存储一个包含每个图像路径和标签的索引文件。
- 索引文件必须是CSV或JSONL格式。

#### CSV

对于图像分类，CSV索引文件需要满足以下要求：

- 无标题。
- 第一列是指向图像的云存储路径。
- 第二列是标签。

#### 云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

#### 快速查看您的数据

您将使用存储在公共云存储桶中的 Flowers 数据集的一个版本，使用一个 CSV 索引文件。

首先快速查看数据。您可以通过计算 CSV 索引文件中的行数（`wc -l`）来统计示例数，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的Vertex Dataset资源中。使用这个辅助函数`import_data`来导入数据。该函数执行以下操作：

- 使用`Dataset`客户端。
- 调用客户端方法`import_data`，并提供以下参数：
 - `name`：您为`Dataset`资源指定的可读名称（例如，flowers）。
 - `import_configs`：导入配置。

- `import_configs`：一个包含字典的Python列表，其键/值条目包括：
 - `gcs_sources`：包含一个或多个索引文件路径的URI列表。
 - `import_schema_uri`：标识标签类型的模式。

`import_data()`方法返回一个长时间运行的`operation`对象。这将需要几分钟才能完成。如果您在进行即时教程，那么现在是一个提问或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在，使用您的 Vertex `Dataset` 资源来训练一个 AutoML 图像分类模型。要训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex 训练流水线。
2. 执行流水线以开始训练。

### 创建一个训练流水线

你可能会问，我们为什么要使用流水线？通常在工作（比如训练）有多个步骤，一般按照顺序进行时，我们会使用流水线：执行步骤A，执行步骤B，等等。通过将这些步骤组合成一个流水线，我们可以获得以下好处：

1. 可以用于后续的训练工作。
2. 可以被容器化并作为批处理作业运行。
3. 可以进行分布式处理。
4. 所有的步骤都与同一个流水线工作相关联，以跟踪进展。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：流水线作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标注（标注）训练模式。
- `task`：描述训练工作要求的字典。

这个辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，它接受以下参数：

- `parent`：您的 `数据集`、`模型` 和 `端点` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线训练作业的详细规范。

现在让我们更深入地看一下构建 `training_pipeline` 规范所需的*最低*要求：

- `display_name`：流水线作业的人类可读名称。
- `training_task_definition`：数据集标注（标注）训练模式。
- `training_task_inputs`：描述训练工作要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：Vertex 数据集标识符（仅限非完全限定）-- 这是完全限定标识符的最后一部分。
 - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比会被 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与接受Python（类似JSON）字典的其他参数不同，`task`字段接受一个Google protobuf Struct，它与Python字典非常相似。使用`json_format.ParseDict`方法进行转换。

我们需要指定的最小字段是：

- `multi_label`：指示这是否为多标签（与单一标签相反）分类的True/False。
- `budget_milli_node_hours`：用于训练模型的最大预算时间（计费时间），其中1000 = 1小时。对于图像分类，预算必须至少为8小时。
- `model_type`：部署模型的类型：
  - `CLOUD`：用于部署到Google Cloud。
  - `MOBILE_TF_LOW_LATENCY_1`：用于部署到边缘并进行延迟（响应时间）优化。
  - `MOBILE_TF_HIGH_ACCURACY_1`：用于部署到边缘并进行准确性优化。
  - `MOBILE_TF_VERSATILE_1`：用于部署到边缘并进行延迟和准确性之间的权衡优化。
- `disable_early_stopping`：指示AutoML是否允许其判断是否提前停止训练或训练整个预算时间的True/False。

最后，通过调用辅助函数`create_pipeline`来创建流水线，该函数返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "flowers_pipe-" + TIMESTAMP
MODEL_NAME = "flowers_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "multi_label": False,
        "budget_milli_node_hours": 8000,
        "model_type": "CLOUD",
        "disable_early_stopping": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取关于训练流水线的信息

现在获取仅此训练流水线实例的流水线信息。通过调用作业客户服务的`get_training_pipeline`方法，使用以下参数获取该作业的作业信息：

- `name`: Vertex完全合格的流水线标识符。

当模型训练完成时，流水线状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，您将需要知道完全限定的Vertex模型资源标识符，即管道服务分配给它的。您可以从返回的管道实例中获取此信息，字段为`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完毕，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为您的模型有多好。作为训练的一部分，数据集的一部分被保留为测试（保留）数据，由流水线服务用于评估模型。

列出所有数据切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`：`Model`资源的Vertex完全限定模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），我们将打印出评估中每个指标的所有键名称，对于一小部分（`logLoss`和`auPrc`），您将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 部署“Model”资源

现在部署您使用AutoML创建的训练后的Vertex“Model”资源。这需要两个步骤：

1. 为部署“Model”资源创建一个“Endpoint”资源。

2. 将“Model”资源部署到“Endpoint”资源。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以提供预测的终端节点，需要传入以下参数：

- `display_name`：`Endpoint`资源的人类可读名称。

辅助函数使用终端节点客户端服务的`create_endpoint`方法，需要传入以下参数：

- `display_name`：`Endpoint`资源的人类可读名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为服务预设`Endpoint`资源。您可以调用`response.result()`，这是一个同步调用，会在`Endpoint`资源准备就绪时返回。辅助函数返回`Endpoint`资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "flowers_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的 `Endpoint` 资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

对于处理您的在线预测请求，您有几种扩展计算实例的选择：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，固定数量的计算实例会被提供，并且在线预测请求会均匀分布在它们之间。

- 自动扩展：在线预测请求被分配到可扩展数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时要提供和取消提供的实例数，并根据负载情况设置最大（`MAX_NODES`）计算实例数。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署`Model`资源到`Endpoint`资源

使用此辅助函数`deploy_model`，将从训练管道中上传（部署）的`Model`资源部署到您为提供预测创建的`Endpoint`资源，具有以下参数：

- `model`：要上传（部署）的模型的Vertex完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的Vertex完全限定终端点标识符。

辅助函数调用`Endpoint`客户端服务的方法`deploy_model`，该方法接受以下参数：

- `endpoint`：要将`Model`资源部署到的Vertex完全限定`Endpoint`资源标识符。
- `deployed_model`：部署模型的要求规范。
- `traffic_split`：指定流量比例在终端点上转到该模型，以字典形式指定一个或多个键/值对。
  - 如果只有一个模型，则指定为**{"0": 100}**，其中"0"指的是上传的模型，100表示100%的流量。
  - 如果有现有模型存在，其流量将被分割，则使用`model_id`指定为**{"0": 百分比，model_id: 百分比，...}**，其中`model_id`是要部署到的终结点的现有模型的模型标识符。百分比必须总计为100。

现在让我们深入研究`deployed_model`参数。此参数作为Python字典指定，具有最小所需字段：

- `model`：要部署的（上传）模型的Vertex完全限定模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这将禁用容器事件的日志记录，例如执行失败（默认情况下容器日志记录已启用）。在调试部署时通常启用容器日志记录，然后在生产部署时禁用。
- `automatic_resources`：这指的是有多少个冗余计算实例（副本）。对于此示例，我们将其设置为一个（不进行复制）。

#### 流量拆分

让我们现在深入研究`traffic_split`参数。此参数作为Python字典指定。这可能一开始会有点混乱。让我解释一下，您可以将您的模型的多个实例部署到一个终端点，并设置每个实例的流量比例。

为什么这样做？也许您已经在生产中部署了之前的版本——让我们称之为v1。您在v2上得到了更好的模型评估，但在部署到生产环境之前，您并不确定它是否真的更好。因此，在流量拆分的情况下，您可能希望将v2部署到与v1相同的终端点，但它只占10%的流量。这样，您可以在不干扰大多数用户的情况下监视其性能，直到最终做出决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`等待操作同步完成，该方法将会阻塞直到模型部署完成。如果这是第一次将模型部署到终端点，可能需要额外几分钟来完成资源的分配。

In [ ]:
DEPLOYED_NAME = "flowers_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出在线预测请求

现在对部署的模型进行在线预测。

获取测试项目

您将从数据集中选择任意一个示例作为测试项目。不必担心该示例很可能用于训练模型 - 我们只是想演示如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

通过云函数代理进行预测

Vertex 中的端点固定在部署的项目中。这意味着通常只有在同一项目内运行的进程才能访问用于预测的端点。

对于基于 web 的应用程序来说，有几种方法可以从端点获取预测结果：

- 基于 web 的应用程序的后端应用服务器部署在同一项目中。

- 后端应用服务器部署在具有对部署端点所在项目授予权限的不同项目中。

- 使用一个代理，该代理具有访问部署端点所在项目的权限。

在此示例中，使用 Cloud Functions 来演示代理方法。

### 创建一个云函数

您首先要创建一个云函数，其中包括：

- 一个文件夹（例如，function）
  - main.py：实现为 Python 脚本的函数。
  - requirements.txt：执行 Python 脚本所需的环境要求。

In [ ]:
! rm -rf function
! mkdir function

In [ ]:
%%writefile function/main.py
import logging
from operator import itemgetter
import os

from flask import jsonify
from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import requests
import tensorflow as tf

IMG_WIDTH = 128
COLUMNS = ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

aip_client = aip.PredictionServiceClient(client_options={
    'api_endpoint': 'us-central1-prediction-aiplatform.googleapis.com'
})
aip_endpoint_name = f'projects/{os.environ["GCP_PROJECT"]}/locations/us-central1/endpoints/{os.environ["ENDPOINT_ID"]}'


def get_prediction(instance):
    logging.info('Sending prediction request to Vertex ...')
    try:
        pb_instance = json_format.ParseDict(instance, Value())
        response = aip_client.predict(endpoint=aip_endpoint_name,
                                      instances=[pb_instance])
        return list(response.predictions[0])
    except Exception as err:
        logging.error(f'Prediction request failed: {type(err)}: {err}')
        return None


def preprocess_image(image_url):
    logging.info(f'Fetching image from URL: {image_url}')
    try:
        image_response = requests.get(image_url)
        image_response.raise_for_status()
        assert image_response.headers.get('Content-Type') == 'image/jpeg'
    except (ConnectionError, requests.exceptions.RequestException,
            AssertionError):
        logging.error(f'Error fetching image from URL: {image_url}')
        return None

    logging.info('Decoding and preprocessing image ...')
    image = tf.io.decode_jpeg(image_response.content, channels=3)
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    image = image / 255.
    return image.numpy().tolist()  # Make it JSON-serializable

def classify_flower(request):
    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows POST requests from any origin with the Content-Type
        # header and caches preflight response for an 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'POST',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }
        return ('', 204, headers)

    # Disallow non-POSTs
    if request.method != 'POST':
        return ('Not found', 404)

    # Set CORS headers for the main request
    headers = {'Access-Control-Allow-Origin': '*'}

    request_json = request.get_json(silent=True)
    if not request_json or not 'image_url' in request_json:
        return ('Invalid request', 400, headers)

    instance = preprocess_image(request_json['image_url'])
    if not instance:
        return ('Invalid request', 400, headers)

    raw_prediction = get_prediction(instance)
    if not raw_prediction:
        return ('Error getting prediction', 500, headers)

    probabilities = zip(COLUMNS, raw_prediction)
    sorted_probabilities = sorted(probabilities,
                                  key=itemgetter(1),
                                  reverse=True)
    return (jsonify(sorted_probabilities), 200, headers)


In [ ]:
%%writefile function/requirements.txt
Flask==1.0.2
requests==2.21.0
tensorflow-cpu~=2.1.0
google-cloud-aiplatform

### 部署您的云函数

**待办事项**
这些 API 需要被启用
启用 Cloud 函数 API 和 Cloud Build API
https://console.developers.google.com/apis/api/cloudbuild.googleapis.com/overview?project=

In [ ]:
! gcloud functions deploy classify_flower
  --region $REGION 
  --source=function 
  --runtime=python37 
  --memory=2048MB 
  --trigger-http 
  --allow-unauthenticated 
  --set-env-vars ENDPOINT_ID=${endpoint_id}

构建部署的云函数的URL

接下来，您将构建云函数的URL。您将使用此URL将预测路由通过云函数，充当您部署的模型的代理。

In [ ]:
CLOUD_FUNCTION_URL = "https://{}-{}.cloudfunctions.net/classify_flower".format(
    REGION, PROJECT_ID
)
print(CLOUD_FUNCTION_URL)

构建预测请求文件

**待办事项**

In [ ]:
image = test_item.replace("gs://", "https://storage.googleapis.com/")
print(image)

import json

with open("request.json", "w") as f:
    json.dump({"image_url": image}, f)

! cat request.json

#### 发出预测请求

**待办**

In [ ]:
! curl -X POST
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) 
-H "Content-Type: application/json; charset=utf-8" 
-d @request.json 
$CLOUD_FUNCTION_URL

取消部署`Model`资源

现在从提供服务`Endpoint`资源中取消部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：在将`Model`资源部署到时由终端服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的 `undeploy_model` 方法，具有以下参数：

- `deployed_model_id`：在将`Model`资源部署时由终端服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上的其余部署模型之间分配流量。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将其设置为空来简单地将`traffic_split`设置为空对象`{}`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

整理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME